# Ingest and Localize

In [ ]:
import wandb
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

ENTITY = "no-organization-for-signup"
PROJECT = "forage-scale-blind"

# Define a function to delete a single run
def export_run(run):
    try:
        # Collect run's summary metrics, configs, and name
        summary = run.summary._json_dict
        config = {k: v for k, v in run.config.items() if not k.startswith('_')}
        name = run.name

        # Combine summary and config into a single dictionary
        run_data = {**summary, **config}
        run_data['name'] = name

        # Optionally add more run metadata
        run_data['id'] = run.id
        run_data['created_at'] = run.created_at
        run_data['state'] = run.state

        hist = run.history()
        hist['id'] = run.id
        hist['steps_pretrained'] = run_data['steps_pretrained']
        hist['num_agents'] = run_data['num_agents']

        return run_data, hist

    except Exception as e:
        return f"Error exporting run {run.id}: {e}"

# Initialize W&B API
api = wandb.Api()
runs = api.runs(f'{ENTITY}/{PROJECT}')

# Initialize lists to hold run data and history
runs_data = []
histories = []

# Set up the ThreadPoolExecutor to parallelize the process
with ThreadPoolExecutor(max_workers=5) as executor:
    # Submit export tasks to the executor
    for run_data, history in executor.map(export_run, runs):
        runs_data.append(run_data)
        histories.append(history)

# Convert the list of dictionaries to a DataFrame
runs_df = pd.DataFrame(runs_data)
hist_df = pd.concat(histories, keys=[f'run_{i}' for i in range(len(histories))])

# Reorder columns so identifying info is at the front
cols = ['id', 'name', 'created_at', 'state'] + \
    [col for col in runs_df.columns if col not in 
     ['id', 'name', 'created_at', 'state']]
runs_df = runs_df[cols]

# Export the DataFrame to CSV
runs_df.to_csv(f"{PROJECT}.csv", index=False)
hist_df.to_csv(f"{PROJECT}_history.csv", index=True)

print(f"Data has been successfully exported to '{PROJECT}.csv'.")

In [ ]:
import numpy as np
import pandas as pd

def baseline_prediction_interval(num_agents, path):
    data = []
    for p in path:
        # Read each table
        _df = pd.read_csv(p)
        # Filter for number of agents
        _df = _df[_df['num_agents']==num_agents]
        # Filter for tabula rasa
        _df = _df[_df['steps_pretrained']==0]
        _df['env_runners/episode_reward_mean'] = (
            _df['env_runners/episode_reward_mean']
            .div(num_agents)
            .replace('NaN',None)
            .bfill())
        data.append(_df)

    seq = (pd.concat(data, ignore_index=True)
           .groupby('_step')['env_runners/episode_reward_mean']
           )

    # Mean and standard deviation per timestep
    mean = seq.mean()
    std_dev = seq.std()
    n = seq.count()

    # Calculate the Prediction Interval (PI)
    # For large n, using z=1.96 for ~95% coverage. (Central limit theorem)
    z = 1.96
    margin_of_error = z * std_dev * np.sqrt(1 + 1/n)

    x = np.concatenate([mean.index]) * num_agents

    return x, mean, margin_of_error


def get_avg_retrain(task_agents, path, pre_agents=2):
    data = []
    for p in path:
        # Read each table
        _df = pd.read_csv(p)
        # Filter for number of agents
        _df = _df[_df['num_agents']==task_agents]
        # Filter for NON tabula rasa
        _df = _df[_df['steps_pretrained']>0]
        #_df = _df[_df['steps_pretrained']<200]
        data.append(_df)

    d1 = (pd.concat(data, ignore_index=True)
          .groupby(['steps_pretrained','_step'])['env_runners/episode_reward_mean']
          .mean()
          .reset_index()
          )
    
    d1['per_agent_erm'] = (d1['env_runners/episode_reward_mean']
                           .div(task_agents)
                           .replace('NaN',None)
                           .bfill())

    d1['timestep'] = pre_agents*d1['steps_pretrained'] + task_agents*d1['_step']

    return d1

import plotly.express as px
import plotly.graph_objects as go

def plot_avg_retrain(task_agents, path, env, pre_agents=2, show=True, write=True):
    retrain_results = get_avg_retrain(task_agents, path)

    base_x, base_mean, base_err = baseline_prediction_interval(task_agents, path)

    fig = px.line(
        retrain_results,
        y="per_agent_erm", 
        x="timestep", color="steps_pretrained", line_group="steps_pretrained",
        color_discrete_sequence=px.colors.qualitative.G10, line_shape="spline", 
        render_mode="svg", 
        title=f"{task_agents} Agent {env} with {pre_agents} Agent Pretraining",
        labels={
            "per_agent_erm" : "Mean Episode Reward per Agent", 
            "timestep": "Agent-steps",
            "steps_pretrained": "Pretraining Length"})

    fig.add_trace(go.Scatter(
        x=base_x,
        y=base_mean-base_err,
        line=dict(color='rgba(255,255,255,0)', width=0),
    ))
    fig.add_trace(go.Scatter(
        x=base_x,
        y=base_mean+base_err,
        fill='tonexty',
        fillcolor='rgba(0,100,80,0.2)',  # semi-transparent fill
        line=dict(color='rgba(255,255,255,0)'),
        name=f'Baseline:<br>{task_agents} Agent 95%<br>Prediction<br>Interval',
        hoverinfo="skip"
    ))

    fig.add_trace(go.Scatter(
        y=base_mean, x=base_x,
        line_color='rgba(0,100,80,0.5)',
        line=dict(dash='dot'),
        name=f'Baseline: Mean'
    ))

    fig.update_layout(width=800, height=500,)
    if show:
        fig.show()
    if write:
        fig.write_image(f"{env}-{task_agents}-agent.png", width=800, height=500)

    return base_x, base_mean, retrain_results